<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/autogen/L1_Multi_Agent_Conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

In [ ]:
!wget https://github.com/JSJeong-me/AutoGen/raw/main/requirements.txt

In [ ]:
!pip install -r requirements.txt

In [ ]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

## Setup

In [ ]:
!wget https://github.com/JSJeong-me/GPT-Agent/blob/main/autogen/utils.py

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    # print(openai_api_key)
    return openai_api_key

# get_openai_api_key()

In [ ]:
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [ ]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "고래를 주제로 재미난 얘기 1문장으로 해줘", "role": "user"}]
)
print(reply)

In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "재미난 이야기 계속해 주세요", "role": "user"}]
)
print(reply)

## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [ ]:
cathy = ConversableAgent(
    name="야옹님",
    system_message=
    "당신의 이름은 야옹님이고 당신은 재미난 이야기를 잘 만드는 코미디언입니다",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="멍멍님",
    system_message=
    "당신의 이름은 멍멍님이고 당신은 재미난 이야기를 잘 만드는 코미디언입니다"
    "이전 대화의 내용을 이어서 다음 재미난 이야기를 시작하세요.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="나는 야옹님이야, 우리 재미난 이야기를 이어서 나가 볼까?",
    max_turns=2,
)

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

In [ ]:
pprint.pprint(chat_result.cost)

In [ ]:
pprint.pprint(chat_result.summary)

## Get a better summary of the conversation

In [ ]:
chat_result = joe.initiate_chat(
    cathy,
    message="나는 멍멍님이야, 우리 재미난 이야기를 이어서 나가 볼까?",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

In [ ]:
pprint.pprint(chat_result.summary)

## Chat Termination

Chat can be terminated using a termination conditions.

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

In [ ]:
cathy.send(message="What's last joke we talked about?", recipient=joe)